In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py
import os
from tqdm import tqdm

from holodeck import plot, detstats, utils
from holodeck.constants import YR, MSOL, MPC, GYR
import holodeck as holo

Set up

In [ ]:
SHAPE = None
NREALS = 500
# NREALS = 20
NFREQS = 40
NLOUDEST = 10

BUILD_ARRAYS = False
SAVEFIG = True
TOL=0.01
MAXBADS=5

NVARS = 21
# NVARS = 6

NPSRS = 40
NSKIES = 100
# NSKIES = 15

MEDIAN=False
AVG=True

Get data and dets file functions

In [ ]:
# def get_data(
#         target, nvars=NVARS, nreals=NREALS, nskies=NSKIES, shape=SHAPE, red_gamma = None, red2white=None,
#     path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz', dsc_flag=False, gsc_flag=False,    
# ):
#     load_data_from_file = path+f'/{target}_v{nvars}_r{nreals}_shape{str(shape)}/data_params.npz' 
#     load_dets_from_file = path+f'/{target}_v{nvars}_r{nreals}_shape{str(shape)}/detstats_s{nskies}' 

#     if gsc_flag: load_dets_from_file = load_dets_from_file + '_gsc'
#     elif dsc_flag: load_dets_from_file = load_dets_from_file + '_dsc'

#     if red_gamma is not None and red2white is not None:
#         load_dets_from_file = load_dets_from_file+f'_r2w{red2white:.1f}_rg{red_gamma:.1f}'
#     else:
#         load_dets_from_file = load_dets_from_file+f'_white'
#     load_dets_from_file = load_dets_from_file+'.npz'

#     if os.path.exists(load_data_from_file) is False:
#         err = f"load data file '{load_data_from_file}' does not exist, you need to construct it."
#         raise Exception(err)
#     if os.path.exists(load_dets_from_file) is False:
#         err = f"load dets file '{load_dets_from_file}' does not exist, you need to construct it."
#         raise Exception(err)
#     file = np.load(load_data_from_file, allow_pickle=True)
#     data = file['data']
#     params = file['params']
#     file.close()
#     print(target, "got data")
#     file = np.load(load_dets_from_file, allow_pickle=True)
#     print(target, "loaded dets")
#     print(file.files)
#     dsdat = file['dsdat']
#     file.close()

#     return data, params, dsdat

In [ ]:
def _twiny_hz(ax, nano=True, fs=10, label=True, **kw):
    tw = ax.twinx()
    tw.grid(False)
    ylim = np.array(ax.get_ylim()) / YR
    if nano:
        ylim *= 1e9

    tw.set(ylim=ylim, yscale=ax.get_yscale())
    if label:
        label = plot.LABEL_GW_FREQUENCY_NHZ if nano else plot.LABEL_GW_FREQUENCY_YR
        tw.set_ylabel(label, fontsize=fs, **kw)
    return tw

# Build arrays using weighted mean and variance

In [ ]:
# def build_freq_arrays(target, gsc_flag=False, dsc_flag=False, nodiv_flag=False):
#     figpath = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata/favg' 
#     xx = [] # param
#     yy = [] # frequency means
#     yy_log = []
#     vv = [] # variance
#     vv_log = [] # variance of log frequency

#     # white noise only
#     data, params, dsdat = get_data(target, gsc_flag=gsc_flag, dsc_flag=dsc_flag)
#     freqs = data[0]['fobs_cents']
#     freqs = np.repeat(freqs, NREALS*NSKIES*NLOUDEST).reshape(NFREQS, NREALS, NSKIES, NLOUDEST)

#     for pp, par in enumerate(params):
#         xx.append(params[pp][target])
#         dpssi = dsdat[pp]['gamma_ssi']
#         mean, var2 = detstats.weighted_mean_variance(freqs, weights=dpssi)
#         logmean, logvar2 = detstats.weighted_mean_variance(np.log10(freqs), weights=dpssi)
        
#         var = np.sqrt(var2)
#         logvar = np.sqrt(logvar2)

#         yy.append(mean)
#         yy_log.append(logmean)
#         vv.append(var)
#         vv_log.append(logvar)
#     saveloc = figpath+f'/freq_meanvar_{target}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}'
#     if gsc_flag: 
#         saveloc = saveloc+'_gsc'
#         if nodiv_flag: saveloc = saveloc+'_nodiv'
#     elif dsc_flag: saveloc = saveloc+'_dsc'
#     saveloc=saveloc+'.npz'
#     np.savez(saveloc, xx = xx, yy = yy, vv = vv, yy_log = yy_log, vv_log=vv_log)
# xx = []
# yy = []

In [ ]:
# def get_freq_arrays(target, 
# gsc_flag=False, dsc_flag=False, nodiv_flag=False):
#     figpath = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata/favg' 
#     saveloc = figpath+f'/freq_meanvar_{target}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}'
#     if gsc_flag: 
#         saveloc = saveloc+'_gsc'
#         if nodiv_flag: saveloc = saveloc+'_nodiv'
#     elif dsc_flag: saveloc = saveloc+'_dsc'
#     saveloc=saveloc+'.npz'

#     file = np.load(saveloc)
#     xx = file['xx']
#     yy = file['yy']
#     vv = file['vv']
#     yy_log = file['yy_log']
#     vv_log = file['vv_log']
#     file.close()

#     return xx, yy, vv, yy_log, vv_log

In [ ]:
# def get_gw_freq_arrays(target, 
# gsc_flag=False, dsc_flag=False, nodiv_flag=False):
#     figpath = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata/favg' 
#     saveloc = figpath+f'/freq_meanvar_gw_{target}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}'
#     if gsc_flag: 
#         saveloc = saveloc+'_gsc'
#         if nodiv_flag: saveloc = saveloc+'_nodiv'
#     elif dsc_flag: saveloc = saveloc+'_dsc'
#     saveloc=saveloc+'.npz'

#     file = np.load(saveloc)
#     xx = file['xx']
#     yy = file['yy']
#     vv = file['vv']
#     yy_log = file['yy_log']
#     vv_log = file['vv_log']
#     file.close()

#     return xx, yy, vv, yy_log, vv_log

# Plot, with Weighted STdev

$$ \bar{X} = \big( \sum_i W_i X_i  \big)/\sum_i W_i$$
$$ S^2 = \big(\sum_i W_i (X_i - \bar{X})^2\big) / \big( \frac{n-1}{n} \sum_i W_i  \big)

# WN Favg

In [ ]:
BUILD_ARRAYS = False
INCLUDE_GW = False

targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
    'mmb_mamp_log10', 'mmb_scatter_dex',
    'hard_time', 'hard_gamma_inner']
colors = [
    '#336948', '#336948', 
    '#9e5c41', '#9e5c41', 
    '#2d839f', '#2d839f',
]   

ylabel = 'Frequency [yr$^{-1}$]'
ylabel_nHz = 'Frequency [nHz]'
fig, axs = plot.figax_single(nrows=3, ncols=2, 
                             sharey=True, sharex=False, xscale='linear',
                             height=7)


for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])
    # if ii == 0 or ii == 3:
    #     ax.set_ylabel(ylabel)
    # file = np.load(path+f'/freq_meanvar_{targets[ii]}_v{NVARS}_r{NREALS}_s{NSKIES}_shape{str(SHAPE)}.npz')
    if BUILD_ARRAYS:
        xx, yy_log, vv_log = holo.detstats.build_favg_arrays(
            targets[ii], bgl=1)
    else:
        xx, yy_log, vv_log = holo.detstats.get_favg_arrays(
            targets[ii], bgl=1)

    yy_min = 10**(yy_log - vv_log) *YR
    yy_max = 10**(yy_log + vv_log) *YR
    yy = 10**yy_log * YR


    if INCLUDE_GW:
        if ii<4:
            if BUILD_ARRAYS:
                xx_gw, yy_log_gw, vv_log_gw = holo.detstats.build_favg_arrays(
                    targets[ii], gw_only=True, bgl=1)
            else:
                xx_gw, yy_log_gw, vv_log_gw = holo.detstats.get_favg_arrays(
                    targets[ii], gw_only=True, bgl=1)
            
            yy_min_gw = 10**(yy_log_gw - vv_log_gw) *YR
            yy_max_gw = 10**(yy_log_gw + vv_log_gw) *YR
            yy_gw = 10**yy_log_gw * YR

        elif ii==4: # hard_time, get mean, hard_gamma_inner uses same
            yy_gw = np.repeat(yy_gw[10], NVARS)
            yy_min_gw = np.repeat(yy_min_gw[10], NVARS)
            yy_max_gw = np.repeat(yy_max_gw[10], NVARS)

    ax.plot(xx, yy, color=colors[ii], alpha=0.9)
    if INCLUDE_GW: 
        ax.plot(xx, yy_gw, color='k', alpha=0.75, linestyle='--')

    ax.fill_between(xx, yy_min, yy_max, color=colors[ii], alpha=0.25, )
    if INCLUDE_GW:
        ax.fill_between(xx, yy_min_gw, yy_max_gw, color='k', alpha=0.15, )
    
    if ii%2 == 1:
        _twiny_hz(ax, label=False)

    # print min and max freq
    # print(f"{np.min(yy)=}, {np.max(yy)=}, {np.min(yy_gw)=}, {np.max(yy_gw)=}")
# fig.tight_layout()
    # ax.tick_params(axis='x', labelrotation=45)

fig.text(0.028, 0.5, ylabel, ha='right', va='center', rotation='vertical')
fig.text(1.095, 0.5, ylabel_nHz, ha='right', va='center', rotation='vertical')
plt.subplots_adjust(wspace=0.05, hspace=0.35)
saveloc = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots/favg'
savename = saveloc+'/favg_gw_single_ci.png'
fig.savefig(savename, dpi=100, bbox_inches='tight')

# Use gsc-flag, nodiv

In [ ]:

# targets = [
#     'gsmf_phi0', 'gsmf_mchar0_log10', 
#     'mmb_mamp_log10', 'mmb_scatter_dex',
#     'hard_time', 'hard_gamma_inner']
# colors = [
#     '#336948', '#336948', 
#     '#9e5c41', '#9e5c41', 
#     '#2d839f', '#2d839f',
# ]
# path = '/Users/emigardiner/GWs/holodeck/output/anatomy_redz/figdata'  
   

# ylabel = 'Frequency [yr$^{-1}$]'
# ylabel_nHz = 'Frequency [nHz]'
# fig, axs = plot.figax_single(nrows=3, ncols=2, 
#                              sharey=True, sharex=False, xscale='linear',
#                              height=7)


# for ii, ax in enumerate(axs.flatten()):
#     ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])
#     # if ii == 0 or ii == 3:
#     #     ax.set_ylabel(ylabel)
#     xx, yy, vv, yy_log, vv_log = get_freq_arrays(targets[ii], gsc_flag=True, nodiv_flag=True)
#     yy_min = 10**(yy_log - vv_log) *YR
#     yy_max = 10**(yy_log + vv_log) *YR
#     yy = 10**yy_log * YR


#     ax.plot(xx, yy, color=colors[ii], alpha=0.9)
#     # ax.plot(xx, yy_gw, color='k', alpha=0.75, linestyle='--')
#     # ax.errorbar(xx, yy, yerr=(yy-yy_min, yy_max-yy), color=colors[ii], alpha=0.7, capsize=3)

#     ax.fill_between(xx, yy_min, yy_max, color=colors[ii], alpha=0.25, )
#     # ax.fill_between(xx, yy_min_gw, yy_max_gw, color='k', alpha=0.15, )
#     # ax.scatter(xx, yy_max_gw, color='k', alpha=0.5, marker=7)
    
#     if ii%2 == 1:
#         _twiny_hz(ax, label=False)

#     # print min and max freq
#     print(f"{np.min(yy)=}, {np.max(yy)=}, {np.min(yy_gw)=}, {np.max(yy_gw)=}")
# # fig.tight_layout()
#     # ax.tick_params(axis='x', labelrotation=45)

# fig.text(0.02, 0.5, ylabel, ha='right', va='center', rotation='vertical')
# fig.text(1.09, 0.5, ylabel_nHz, ha='right', va='center', rotation='vertical')
# plt.subplots_adjust(wspace=0.05, hspace=0.35)
# saveloc = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots'
# savename = saveloc+'/fmeanvar_gsc_single_ci.png'
# fig.savefig(savename, dpi=100, bbox_inches='tight')

# Red Noise

In [ ]:
cmap_Oranges = plot.truncate_colormap('Oranges', 0.4, 1)
cmap_Reds = plot.truncate_colormap('Reds', 0.4, 1)
cmap_Purples = plot.truncate_colormap('Purples', 0.4, 1)
cmap_Greys = plot.truncate_colormap('Greys', 0.4, 0.9)

In [ ]:
# BUILD_ARRAYS = True
# if BUILD_ARRAYS:
#     for target in [
#         'hard_time', 'gsmf_phi0', 'gsmf_mchar0_log10',
#         'mmb_mamp_log10', 'mmb_scatter_dex', 'hard_gamma_inner'
#     ]:
#         for red2white in [0.5, 1.0, 2.0]:
#             for red_gamma in [-1.5, -3.0]:
#                 detstats.build_freq_arrays(
#                     target, red2white=red2white, red_gamma=red_gamma,
#                     gw_only=False)

In [ ]:
def draw_favg(ax, xx, yy_log, vv_log, 
              color=None, linestyle=None, alpha=0.9,
              sigmas = False):
    yy = 10**yy_log * YR
    hh, = ax.plot(xx, yy, color=color, linestyle=linestyle, alpha=alpha)
    if sigmas:
        yy_min = 10**(yy_log - vv_log) *YR
        yy_max = 10**(yy_log + vv_log) *YR
        ax.fill_between(xx, yy_min, yy_max, color=color, alpha=0.15, )
    return hh



In [ ]:
import matplotlib as mpl

In [ ]:
BUILD_ARRAYS = False
INCLUDE_GW = False

targets = [
    'gsmf_phi0', 'gsmf_mchar0_log10', 
    'mmb_mamp_log10', 'mmb_scatter_dex',
    'hard_time', 'hard_gamma_inner']
slope_colors = [
    'black',   
]  
color_wn = 'black' 
lines_wn = ':'
cmaps_rn = [cmap_Oranges, cmap_Purples] #, cmap_Purples]
lines_rn = ['-.', '--', '-']

gammas = [ -3.0]
red2whites = [0.5, 1.0, 10.0]

ylabel = 'Frequency [yr$^{-1}$]'
ylabel_nHz = 'Frequency [nHz]'
fig, axs = plot.figax_single(nrows=3, ncols=2, 
                             sharey=True, sharex=False, xscale='linear',
                             height=7)

handles_gamma = []
labels_gamma = []

handles_red2white = []
labels_red2white = []
for ii, ax in enumerate(axs.flatten()):
    ax.set_xlabel(plot.PARAM_KEYS[targets[ii]])

    xx, yy_log, vv_log = holo.detstats.get_favg_arrays(
        targets[ii], bgl=1)

    draw_favg(ax, xx, yy_log, vv_log, color=color_wn, linestyle=lines_wn,
              sigmas=True)
    yy_min = 10**(yy_log - vv_log) *YR
    yy_max = 10**(yy_log + vv_log) *YR
    yy = 10**yy_log * YR
    ax.plot(xx, yy, color=color_wn, linestyle=lines_wn, alpha=0.9)

    for gg, gamma in enumerate(gammas):
        colors = cmaps_rn[gg](np.linspace(0,1,len(red2whites)))
        for rr, red2white in enumerate(red2whites):
            xx, yy_log, vv_log = holo.detstats.get_favg_arrays(
                targets[ii], bgl=1, red_gamma=gamma, red2white=red2white)
            hh = draw_favg(ax, xx, yy_log, vv_log, color=colors[rr], linestyle=lines_rn[rr],
              sigmas=True)
            if ii==0 and lines_rn[rr] == '--':
                handles_gamma.append(hh)
                labels_gamma.append(f"{gamma:.1f}")
                print(gg,rr, colors[rr])
            if ii==0 and gg==0:
                handles_red2white.append(hh)
                labels_red2white.append(f"{red2white:.1f}")


    if ii%2 == 1:
        _twiny_hz(ax, label=False)

    # print min and max freq
    # print(f"{np.min(yy)=}, {np.max(yy)=}, {np.min(yy_gw)=}, {np.max(yy_gw)=}")
# fig.tight_layout()
    # ax.tick_params(axis='x', labelrotation=45)

leg1 = fig.legend(handles=handles_gamma, labels=labels_gamma, title='$\gamma_\mathrm{RN}$',
                  bbox_to_anchor=(0.2,-0.02), loc='lower left', ncols=len(handles_gamma))
leg2 = fig.legend(handles=handles_red2white, labels=labels_red2white, title='$A_\mathrm{RN}/A_\mathrm{WN}$',
                  bbox_to_anchor=(0.55, -0.02), loc='lower left', ncols=len(handles_red2white))

fig.text(0.028, 0.5, ylabel, ha='right', va='center', rotation='vertical')
fig.text(1.095, 0.5, ylabel_nHz, ha='right', va='center', rotation='vertical')
plt.subplots_adjust(wspace=0.05, hspace=0.35)
saveloc = '/Users/emigardiner/GWs/holodeck/output/figures/bigplots/favg'
savename = saveloc+'/favg_rn_r2w0.5_2.0.png'
fig.savefig(savename, dpi=100, bbox_inches='tight')

# Test red2white

In [ ]:
data, params, = detstats.get_data('gsmf_phi0', bgl=1, dets=False)
var=10

fobs_cents = data[var]['fobs_cents']
dur = 1.0 / fobs_cents[0]
cad = 1.0 / (2.0 * fobs_cents[-1])
hc_ss = data[var]['hc_ss']
hc_bg = data[var]['hc_bg']

In [ ]:
import hasasia.sim as hsim

In [ ]:
phis = None
thetas = None
npsrs = 40
red2white = 1.0
red_gamma = -3.0

rr=0
psrs, red_amp, _sigstart, _sigmin, _sigmax = detstats.calibrate_one_pta(hc_bg[:,rr], hc_ss[:,rr,:], fobs_cents, npsrs, 
                                    debug=True, ret_sig=True)
_sigmin /= 2
_sigmax *= 2

# get background detstats
_dp_bg, _snr_bg = detstats.detect_bg_pta(psrs, fobs_cents, hc_bg[:,rr:rr+1], ret_snr=True)
print(_dp_bg)

In [ ]:
phis = None
thetas = None
npsrs = 40
red2white = 100000
red_gamma = -3.0

rr=0
psrs, red_amp, _sigstart, _sigmin, _sigmax = detstats.calibrate_one_pta(hc_bg[:,rr], hc_ss[:,rr,:], fobs_cents, npsrs, 
                                    debug=True, ret_sig=True)
_sigmin /= 2
_sigmax *= 2

# get background detstats
_dp_bg, _snr_bg = detstats.detect_bg_pta(psrs, fobs_cents, hc_bg[:,rr:rr+1], ret_snr=True, red_amp=red_amp, red_gamma=red_gamma)
_dp_ss, _snr_ss, _gamma_ssi = detstats.detect_ss_pta(
            psrs, fobs_cents, hc_ss[:,rr:rr+1], hc_bg[:,rr:rr+1], 
            nskies=NSKIES, ret_snr=True, red_amp=red_amp, red_gamma=red_gamma)


print(f"{_dp_bg=}, {np.max(_dp_ss)=}")

nfreqs = len(fobs_cents)
freqs = np.repeat(fobs_cents*YR, NSKIES*NLOUDEST).reshape(
    nfreqs, NSKIES, NLOUDEST)

logmean, logvar2 = detstats.weighted_mean_variance(np.log10(freqs), weights=_gamma_ssi)
print(10**(logmean))